# Module `reducedbasis`

## Import

*Note:* The work has not yet been merged in `develop`. To include the module at current state, you need to checkout the branch [`feature/pyrb`](https://github.com/feelpp/feelpp/tree/feature/pyrb), and move to the directory [mor/pyfeelpp-mor/feelpp/mor](https://github.com/feelpp/feelpp/tree/feature/pyrb/mor/pyfeelpp-mor/feelpp/mor).
Line to import the module :


In [0]:
from reducedbasis.reducedbasis import *

To set the environment, those module also need to be imported :


In [0]:
import sys
from feelpp.toolboxes.heat import *
from feelpp.toolboxes.core import *
from feelpp.mor import *
import feelpp

## Set the model

To declare the objects, use these lines :


In [0]:
DIM = 2
heatBox=heat(dim=DIM,order=1)
heatBox.init()
model = toolboxmor(dim=DIM, time_dependent=False)
model.setFunctionSpaces( Vh=heatBox.spaceTemperature())


- `DIM` is the dimension of the case (should be `2` or `3`)


- `time_dependent` depends if the case is time-steady or time-dependant.

Those functions define how the `DEIM` matrix and right-hand side are updated according to the parameter `mu`.


In [0]:
def assembleDEIM(mu):
    for i in range(0,mu.size()):
        heatBox.addParameterInModelProperties(mu.parameterName(i),mu(i))
    heatBox.updateParameterValues()
    return heatBox.assembleRhs()

def assembleMDEIM(mu):
    for i in range(0,mu.size()):
        heatBox.addParameterInModelProperties(mu.parameterName(i),mu(i))
    heatBox.updateParameterValues()
    return heatBox.assembleMatrix()

model.setAssembleDEIM(fct=assembleDEIM)
model.setAssembleMDEIM(fct=assembleMDEIM)
model.initModel()

*Warning:* When the function `assembleDEIM` is called, the value of the parameters of the object `heatBox` is updated.
Creation of `DEIM` and `MDEIM` toolboxes, with the associated assembly functions.


In [0]:
heatBoxDEIM=heat(dim=DIM,order=1)
meshDEIM = model.getDEIMReducedMesh()
heatBoxDEIM.setMesh(meshDEIM)
heatBoxDEIM.init()

def assembleOnlineDEIM(mu):
    for i in range(0,mu.size()):
        heatBoxDEIM.addParameterInModelProperties(mu.parameterName(i),mu(i))
    heatBoxDEIM.updateParameterValues()
    return heatBoxDEIM.assembleRhs()

model.setOnlineAssembleDEIM(assembleOnlineDEIM)

heatBoxMDEIM=heat(dim=DIM,order=1)
meshMDEIM = model.getMDEIMReducedMesh()
heatBoxMDEIM.setMesh(meshMDEIM)
heatBoxMDEIM.init()

def assembleOnlineMDEIM(mu):
    for i in range(0,mu.size()):
        heatBoxMDEIM.addParameterInModelProperties(mu.parameterName(i),mu(i))
    heatBoxMDEIM.updateParameterValues()
    return heatBoxMDEIM.assembleMatrix()

model.setOnlineAssembleMDEIM(assembleOnlineMDEIM)

Then finalize the initialization :


In [0]:
model.postInitModel()
model.setInitialized(True)

## Access to a parameter



In [0]:
Dmu = model.parameterSpace()
mu = Dmu.element(True, False)

See  for the API of `ParameterSpaceElement`
